<a href="https://colab.research.google.com/github/OGsiji/Transrisk-Innovation/blob/main/classification_of_time_series_with_lstm_rnn_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Problem description
--------------------------
I've got this dataset of financial time series from my freinds at TenViz who's job is the magic of predicting stock market movements. The problem was formulated as follows:
> ... to predict *target_class* based on values of *target_variable* & available features (dataset 'test_task_data.csv'). Before building a classifier, please pay attention to the nature of features and specific aspects of working with time series. Also, you can use *target_values* to derive useful information and additional features. As a train set use dataset from 2012-01-01 till 2016-12-31, as a test set used from 2017-01-02 till 2018-06-19. Finally, evaluate your model & provide analysis with short comments.
> The results of the work should contain:
> * Description of the steps of the solution of the task.
> * Runnable implementation code in Python.
> * PDF with the charts

In this kernel/ notebook I review the raw time series data, apply necessery transformations and scaling, formulate a machine learning problem and build a classifier based on a stacked LSTM RNN.

## Contents:
1. [Load and Review Data](#1)
2. [Feature Engineering](#2)
3. [Data Pre-processing for LSTM Model](#3)
4. [LSTM Model - Batch Training and Predictiction](#4)
5. [APPENDIX - EDA for Individual Time Series](#5)

In [ ]:
# Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap

%matplotlib inline

# <a name="1"></a> 1. Load and Review Data
------------------

In [ ]:
# Import data, convert string dates to 'datetime64' and set the date column as index:
df = pd.read_csv('../input/mainnnn/mainnn.csv',
                 parse_dates=['DATE'],
                 infer_datetime_format=True,
                 index_col='DATE',
                 thousands=',',
                 decimal='.'
                )

In [ ]:
#  Review the general info on data, paying attention to missing values and dtypes
df.shape

In [ ]:
# Let's remove the empty column and look at some examples of data:
#df = df.drop(columns='Unnamed: 17')
#print(f'data shape = {df.shape}')
#df.head()

In [ ]:
# It appears that 'feature_5' has missing values up to 2012-10-18
# let's fill them backwards
df = df.fillna(method='bfill')

In [ ]:
import pandas as pd

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)


df=clean_dataset(df)


In [ ]:
# Basic statistics of the data:
df.shape

In [ ]:
# Plot the time series
plt.style.use('fivethirtyeight')
df.plot(subplots=True,
        layout=(6, 4),
        figsize=(22,22),
        fontsize=10, 
        linewidth=2,
        sharex=False,
        title='Visualization of the original Time Series')
plt.show()

In [ ]:
# Let's also draw a heatmap visualization of the correlation matrix
corr_matrix = df.corr(method='spearman')
f, ax = plt.subplots(figsize=(16,8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', linewidth=0.4,
            annot_kws={"size": 10}, cmap='coolwarm', ax=ax)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

# <a name="2"></a> 2. Feature Engineering
-------
### What do we have from the raw data review?
* Exploratory Data Analysis of individual time series is visualized in the [APPENDIX](#index) below.
* The raw data contain stochastic time series, including 'target_value'. Predicting/ making classification based on stochastic variable values may force the model to learn the 'persistence' mode (i.e. yhat(t+1) = y(t)), resulting in little predictive power. Defining the model to predict (make classification from) the difference in values between the time steps rather than value itself, is a stronger test of its predictive power. 
* The raw data are weakly correleted with the target_value and the target_class and among each other with rare exceptions. The predictive power could be in temporal effects. 
* The raw data are at different scales, therefore one more review after data transformation is needed to check if a standartization (for ~normal distribution of values) / normalization (for other distributions) is required to ensure efficient learning of NN models.

Possible nature | TS_name | Description | Transformation | Rescaling after transformation
:---: | :-- | :-- | :-- | :--
class | target_class | binary (48%-1s, 52%-0s) | None | None
stock index or indicator like RSI | [target_value](#target_value) | unimodal, bell-shaped, skewed to the right, stochastic trend with values in [48, 91], order +2 autocorr | pc_change or log diff | Standard
stock index | [feature_1](#feature_1) | unimodal, bell-shaped, slightly skewed to the right, stochastic trend with values in [1432, 2539], order +2 autocorr | pc_change or log diff | Standard
stock index | [feature_2](#feature_2) | bimodal, stochastic trend with values in [27, 126 ], order +2 autocorr | pc_change or log diff | Standard or MinMax
stock index | [feature_3](#feature_3) | unimodal, strongly skewed to the right,  stochastic trend with values in [315, 830], order +2 autocorr  | pc_change or log diff | Standard or MinMax
stock index | [feature_4](#feature_4) | unimodal, skewed to the right,  stochastic trend with values in [1, 6 ], order +2 autocorr | pc_change or log diff | Standard
technical indicator | [feature_5](#feature_5) | unimodal, 5 descrete values  in [100+-0.00002], ordr +2 autocorr |  pc_change or log diff | Standard
technical indicator | [feature_6](#feature_6) | unimodal, bell-shaped, ranging in [100+- 0.04], order +2 autocorr | pc_change or log diff | Standard
technical indicator | [feature_7](#feature_7) | unimodal, skewed to the right, ranging in [100+-0.04], order -29/+31 autocorr  | pc_change or log diff | Standard
technical indicator | [feature_8](#feature_8) | ranging around 100, occasional spikes in range [-340, +780] - [Q1'12, Q4'16], order 2 autocorr, negative values! | pc_change | Standard or MinMax
technical indicator | [feature_9](#feature_9) | ranging around 100, occasional spikes in range [-3413, +2626] - [Q1'12, Q4'16], order 2 autocorr, negative values! | pc_change | Standard or MinMax
technical indicator | [feature_10](#feature_10) | ranging around 100, occasional spikes in range [-2104, +2206] - [Q1'12, Q4'16], order 2 autocorr, negative values! | pc_change | Standard or MinMax
technical indicator | [feature_11](#feature_11) | ranging around 100, occasional spikes in range [-1321, +1213 ] - [Q4'12-Q1'13], order 2 autocorr, negative values! | pc_change | Standard or MinMax
technical indicator | [feature_12](#feature_12) | ranging around 100, occasional spikes in range [-2933, +2462 ] - [Q4'12-Q1'13], order 2 autocorr, negative values! | pc_change | Standard or MinMax
technical indicator | [feature_13](#feature_13) | ranging around 100, occasional spikes in range [-3206, +2687 ] - [Q4'12-Q1'13], order 2 autocorr, negative values! | pc_change | Standard or MinMax
technical indicator (oscillator) | [feature_14](#feature_14) | unimodal, skewed to the right, ranging in  [100+-0.02], autocorr +32 | pc_change or log diff | Standard
technical indicator (oscillator) | [feature_15](#feature_15) | unimodal, bell-shaped, skewed to the left, ranging in [100+-0.02], autocorr -38 | pc_change or log diff | Standard
technical indicator (oscillator) | [feature_16](#feature_16) | unimodal, bell-shaped, ranging in [100+-0.01], autocorr +14 | pc_change or log diff | Standard

### Let's try a simple approach first and apply the pc_change to all *features* and *target_value* time series.

In [ ]:
# We want to keep original time series for the EDA in APPENDIX
# So we apply the 'pc_change()' transformation to a copy of the original time series
#df_transform = df[:,:'Target'].copy().pct_change(1)
#df_transform.head()

In [ ]:
#df.head()

In [ ]:
# fill the 1st row with NA data
#df_transform.fillna(method='bfill', inplace=True)
#df_transform.shape

In [ ]:
# Plot the transformed time series
plt.style.use('fivethirtyeight')
df_transform.plot(subplots=True,
                  layout=(6, 4),
                  figsize=(24,24),
                  fontsize=10, 
                  linewidth=2, 
                  title='Visualization of the transformed Features')
plt.show()

In [ ]:
df.head()

### (!) We see that the mean and variance look almost constant except for a few outliers, but it is better to rescale the transformed time series either with StandardScaler or MinMaxScaler(-1, 1) for efficient model learning. The scalers are fit on the traning data and then applied to the test data.

In [ ]:
# Split train and test data  
train_features = df.loc['2013-11-03':'2017-03-07']  
train_labels = df.loc['2013-11-03':'2017-03-07', 'Target']

test_features = df.loc['2018-05-25':'2018-12-11']
test_labels = df.loc['2018-05-25':'2018-12-11', 'Target']




# I want to use a T-days window of input data for predicting target_class
# It means I need to prepend (T-1) last train records to the 1st test window
T = 1 # my choice of the timesteps window

prepend_features = train_features.iloc[-(T-1):]
test_features = pd.concat([prepend_features, test_features], axis=0)

train_features.shape, train_labels.shape, test_features.shape, test_labels.shape

In [ ]:
# Rescale the features
from sklearn.preprocessing import StandardScaler  # MinMaxScaler

scaler = StandardScaler()  # MinMaxScaler(feature_range=(-1,1))
scaled_train_features = pd.DataFrame(scaler.fit_transform(train_features.values),
                                     index=train_features.index,
                                     columns=train_features.columns)
# The Scaler is fit on the training set and then applied to the test set
scaled_test_features = pd.DataFrame(scaler.transform(test_features.values),
                                    index=test_features.index,
                                    columns=test_features.columns)

scaled_train_features.shape, scaled_test_features.shape

In [ ]:
# Plot the rescaled_train_features
plt.style.use('fivethirtyeight')
scaled_train_features.plot(subplots=True,
                           layout=(6, 4),
                           figsize=(24,24),
                           fontsize=10, 
                           linewidth=2, 
                           title='Visualization of the scaled Train Features')
plt.show()

>  ## An alternative simplified transformation of the time-series (just to keep in mind): 
> Normalize sequences X = X/X_0 - 1, where X_0 is 1st example in the series:
> * X_0_train, X_0_test = X_train[ : , 0], X_test[ : , 0]
> * X_train = X_train/X_0_train[ : , None, :] - 1
> * X_test = X_test/X_0_test[ : , None, :] - 1

# <a name="3"></a> 3. Data Pre-processing for LSTM Model
------------------------------------
Input data for the Keras LSTM layer has 3 dimensions: (M, T, N), where 
* M - number of examples (2D: sequences of timesteps x features), 
* T - sequence length (timesteps) and 
* N - number of features (input_dim)

In [ ]:
# Create sequences of T timesteps

X_train, y_train = [], []
for i in range(train_labels.shape[0] - (T-1)):
    X_train.append(scaled_train_features.iloc[i:i+T].values)
    y_train.append(train_labels.iloc[i + (T-1)])
X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1,1)
print(f'Train data dimensions: {X_train.shape}, {y_train.shape}')

X_test, y_test = [], []
for i in range(test_labels.shape[0]):
    X_test.append(scaled_test_features.iloc[i:i+T].values)
    y_test.append(test_labels.iloc[i])
X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1,1)  

print(f'Test data dimensions: {X_test.shape}, {y_test.shape}')

In [ ]:
#train_labels.shape

In [ ]:
#import numpy as np
#X_train = np.reshape(train_labels, (train_labels.shape[0], 1, train_labels.shape[1]))
#X_test = np.reshape(train_labels, (train_labels.shape[0], 1, train_labels.shape[1]))



# <a name="4"></a> 4. LSTM Model - Batch Training and Predictiction
-------------------------

In [ ]:
# Import Keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time

In [ ]:
# Let's make a list of CONSTANTS for modelling:
LAYERS = [8, 8, 8, 1]                # number of units in hidden and output layers
M_TRAIN = X_train.shape[0]           # number of training examples (2D)
M_TEST = X_test.shape[0]             # number of test examples (2D),full=X_test.shape[0]
N = X_train.shape[2]                 # number of features
BATCH = M_TRAIN                          # batch size
EPOCH = 50                           # number of epochs
LR = 5e-2                            # learning rate of the gradient descent
LAMBD = 3e-2                         # lambda in L2 regularizaion
DP = 0.0                             # dropout rate
RDP = 0.0                            # recurrent dropout rate
print(f'layers={LAYERS}, train_examples={M_TRAIN}, test_examples={M_TEST}')
print(f'batch = {BATCH}, timesteps = {T}, features = {N}, epochs = {EPOCH}')
print(f'lr = {LR}, lambda = {LAMBD}, dropout = {DP}, recurr_dropout = {RDP}')

# Build the Model
model = Sequential()
model.add(LSTM(input_shape=(T, N), units=LAYERS[0],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[1],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[2],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=False, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(Dense(units=LAYERS[3], activation='sigmoid'))

# Compile the model with Adam optimizer
model.compile(loss='binary_crossentropy',
              metrics=['accuracy'],
              optimizer=Adam(lr=LR))
print(model.summary())

# Define a learning rate decay method:
lr_decay = ReduceLROnPlateau(monitor='loss', 
                             patience=1, verbose=0, 
                             factor=0.5, min_lr=1e-8)
# Define Early Stopping:
early_stop = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=30, verbose=1, mode='auto',
                           baseline=0, restore_best_weights=True)
# Train the model. 
# The dataset is small for NN - let's use test_data for validation
start = time()
History = model.fit(X_train, y_train,
                    epochs=EPOCH,
                    batch_size=BATCH,
                    validation_split=0.0,
                    validation_data=(X_test[:M_TEST], y_test[:M_TEST]),
                    shuffle=True,verbose=0,
                    callbacks=[lr_decay, early_stop])
print('-'*65)
print(f'Training was completed in {time() - start:.2f} secs')
print('-'*65)
# Evaluate the model:
train_loss, train_acc = model.evaluate(X_train, y_train,
                                       batch_size=M_TRAIN, verbose=0)
test_loss, test_acc = model.evaluate(X_test[:M_TEST], y_test[:M_TEST],
                                     batch_size=M_TEST, verbose=0)
print('-'*65)
print(f'train accuracy = {round(train_acc * 100, 4)}%')
print(f'test accuracy = {round(test_acc * 100, 4)}%')
print(f'test error = {round((1 - test_acc) * M_TEST)} out of {M_TEST} examples')

# Plot the loss and accuracy curves over epochs:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(18,6))
axs[0].plot(History.history['loss'], color='b', label='Training loss')
axs[0].plot(History.history['val_loss'], color='r', label='Validation loss')
axs[0].set_title("Loss curves")
axs[0].legend(loc='best', shadow=True)
axs[1].plot(History.history['acc'], color='b', label='Training accuracy')
axs[1].plot(History.history['val_acc'], color='r', label='Validation accuracy')
axs[1].set_title("Accuracy curves")
axs[1].legend(loc='best', shadow=True)
plt.show()

In [ ]:
y_hat = model.predict_classes(X_test, batch_size=M_TEST, verbose=1)
#score = sum(y_hat == y_test) / len(y_test)
#print(f'Prediction accuracy = {score*100}%')
index = pd.date_range(start='2017-01-02', end='2018-06-19', freq='B')
for i in range(y_hat.shape[0]):
    print(index[i], y_hat[i], y_test[i])

# <a name="5"></a> 5. APPENDIX - Exploratory Data Analysis for Individual Time Series
### <a name="index"></a> INDEX:

Reference | Reference | Reference
:-- | :-- | :--
[EDA Function](#eda_function) | [A-5. feature_5](#feature_5) | [A-11. feature_11](#feature_11) 
[A-0. target_value](#target_value) | [A-6. feature_6](#feature_6)  | [A-12. feature_12](#feature_12) 
[A-1. feature_1](#feature_1) | [A-7. feature_7](#feature_7) | [A-13. feature_13](#feature_13) 
[A-2. feature_2](#feature_2) | [A-8. feature_8](#feature_8) | [A-14. feature_14](#feature_14) 
[A-3. feature_3](#feature_3) | [A-9. feature_9](#feature_9) | [A-15. feature_15](#feature_15) 
[A-4. feature_4](#feature_4) | [A-10. feature_10](#feature_10) | [A-16. feature_16](#feature_16) 

<h1 id="eda_function"> EDA Function </h1>

In [ ]:
# Let's define a EDA function for repeated calls on individual time series:

import statsmodels.api as sm  # seasonal trend decomposition
from statsmodels.graphics import tsaplots   # autocorrelation

def eda(df_name, ts_name, decomp_model='additive'):
    """ 
    Inputs: df_name - name of the dataframe
            ts_name - name of the time series in the dataframe
            decomp_model - 'additive'/'multiplicative'
    Outputs: EDA statistics and plots for individual time series in df_name
    """
    # Statistics
    print(f'Statistic of {ts_name} time series')
    print(df_name[ts_name].describe())
    
    # Plotting
    fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(24,24))
    fig.suptitle(f'Visualization of the "{ts_name}" time series', fontsize=24)
        
    # Observed values of the time series against target_class values
    df_name[ts_name].plot(ylim=[df_name[ts_name].min(), df_name[ts_name].max()],
                          linewidth=2, ax=axs[0,0])
    axs[0,0].set_title('Observed values (red/green where target_class=0/1)')
    axs[0,0].set_xlabel('')
    axs[0,0].fill_between(df_name.index, df_name[ts_name], 
                          where=(df_name.Target==0),
                          facecolor='red', alpha=0.5)
    axs[0,0].fill_between(df_name.index, df_name[ts_name], 
                          where=(df_name.Target==1),
                          facecolor='green', alpha=0.5)
    axs[0,0].axvline('2017-01-01', color='red', linestyle='dashed')
    
    # Seasonality, trend and noise in time series data
    decomp = sm.tsa.seasonal_decompose(df_name[ts_name],
                                       model=decomp_model)
    decomp.trend.plot(linewidth=2, ax=axs[0,1])
    axs[0,1].set_title('Trend values')
    axs[0,1].set_xlabel('')
    decomp.seasonal.plot(linewidth=2, ax=axs[1,0])
    axs[1,0].set_title('Seasonal values')
    axs[1,0].set_xlabel('')
    decomp.resid.plot(linewidth=2, ax=axs[1,1])
    axs[1,1].set_title('Residual values')
    axs[1,1].set_xlabel('')
    
    # Distribution of values of time series
    df_name[ts_name].plot.hist(bins=30, ax=axs[2,0])
    axs[2, 0].set_title('Histogram')
    df_name[[ts_name]].boxplot(ax=axs[2,1])
    axs[2, 1].set_title('Boxplot')
        
    # Autocorrelation of time series
    tsaplots.plot_acf(df_name[ts_name], lags=40, ax=axs[3,0])
    tsaplots.plot_pacf(df_name[ts_name], lags=40, ax=axs[3,1])
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# <a name="target_value"></a> A-0. target_value 
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'Turns', 'multiplicative')


### Conclusion: 
* **target_value** is unimodal bell-shaped (slightly skewed to the right); 
* decomposed into trend and stochastic noise;
* percentage change is likely to have stationarity property
* alternative - log transformation of the variable
* it is possible to apply z-score standartization (sklearn StandardScaler)

# <a name="feature_1"></a> A-1. feature_1
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_1', 'multiplicative')

# <a name="feature_2"></a> A-2. feature_2
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_2', 'multiplicative')

# <a name="feature_3"></a> A-3. feature_3
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_3', 'multiplicative')

# <a name="feature_4"></a> A-4. feature_4
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_4', 'multiplicative')

# <a name="feature_5"></a> A-5. feature_5
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_5')

# <a name="feature_6"></a> A-6. feature_6
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_6')

# <a name="feature_7"></a> A-7. feature_7
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_7')

# <a name="feature_8"></a> A-8. feature_8
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_8')

# <a name="feature_9"></a> A-9. feature_9
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_9')

# <a name="feature_10"></a> A-10. feature_10
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_10')

# <a name="feature_11"></a> A-11. feature_11
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_11')

# <a name="feature_12"></a> A-12. feature_12
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_12')

# <a name="feature_13"></a> A-13. feature_13
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_13')

# <a name="feature_14"></a> A-14. feature_14
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_14')

# <a name="feature_15"></a> A-15. feature_15
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_15')

# <a name="feature_16"></a> A-16. feature_16
[Back to INDEX](#index)

In [ ]:
# Call EDA function to explore the time series
eda(df, 'feature_16')